<a href="https://colab.research.google.com/github/Anushreesharma23/Knowledge-Retrieval-RAG/blob/main/FinalRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install necessary libraries
!pip install streamlit pyngrok llama_index
!pip install llama_index.embeddings.huggingface
!pip install llama_index.llms.llama_cpp
!pip install transformers
!pip install ngrok pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 1

In [2]:
from google.colab import files
import os

# Define the directory to save uploaded files
pdf_directory = '/content/sample_data'

# Create the directory if it doesn't exist
os.makedirs(pdf_directory, exist_ok=True)

# Function to upload files
def upload_files():
    uploaded_files = files.upload()
    for filename in uploaded_files.keys():
        file_path = os.path.join(pdf_directory, filename)
        with open(file_path, 'wb') as f:
            f.write(uploaded_files[filename])
    print(f"Uploaded files to {pdf_directory}")

# Upload files
upload_files()

Saving Research Paper 0901CD211010.pdf to Research Paper 0901CD211010.pdf
Uploaded files to /content/sample_data


In [3]:
#/* Custom CSS for Streamlit app */
%%writefile style.css
body {
    background-color: #f0f2f6;
    color: #333;
    font-family: 'Arial', sans-serif;
}

header {
    background-color: #4CAF50;
    padding: 20px;
    text-align: center;
    font-size: 28px;
    color: white;
    border-radius: 8px;
    margin-bottom: 20px;
}

footer {
    background-color: #4CAF50;
    padding: 15px;
    text-align: center;
    font-size: 50px;
    color: white;
    position: relative;
    bottom: 0;
    width: 100%;
    border-radius: 8px 8px 0 0;
    margin-top: 20px;
}

h1, h2, h3, h4, h5, h6 {
    color: #4CAF50;
}

.stButton>button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 12px 24px;
    font-size: 16px;
    cursor: pointer;
    border-radius: 4px;
    transition: background-color 0.3s ease;
}

.stButton>button:hover {
    background-color: #45a049;
}

.stTextInput>div>div>input {
    padding: 10px;
    border: 2px solid #4CAF50;
    border-radius: 4px;
}

.stFileUploader>div>div>div>button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
    border-radius: 4px;
    transition: background-color 0.3s ease;
}

.stFileUploader>div>div>div>button:hover {
    background-color: #45a049;
}

.stAlert {
    border-left: 5px solid #4CAF50;
    padding: 10px;
    border-radius: 4px;
}

.stSuccess {
    background-color: #e7f6e5;
    border-left: 5px solid #4CAF50;
    padding: 10px;
    border-radius: 4px;
}


Writing style.css


In [4]:
!ngrok config add-authtoken '2j9MNrMbeZEpvOcfinlFoOaatLS_7qwJiUjjX4P1tPGgrcb7P'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
%%writefile app.py
import os
import time
import streamlit as st
from google.colab import files
from llama_index.core import Prompt, Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

# Define the embedding model and LLM URL
text_embedding_model = 'thenlper/gte-base'
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'

# Define the directory to save uploaded files
pdf_directory = '/content/sample_data'

# Create the directory if it doesn't exist
if not os.path.exists(pdf_directory):
    os.makedirs(pdf_directory)

# Function to load and process PDF
def load_and_process_pdf(pdf_directory, text_embedding_model, llm_url):
    loader = SimpleDirectoryReader(input_dir=pdf_directory, recursive=True)
    documents = loader.load_data(num_workers=3)

    embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
    llm = LlamaCPP(
        model_url=llm_url,
        temperature=0.5,
        max_new_tokens=256,
        context_window=4096,
        generate_kwargs={"stop": ["<s>", "[INST]", "[/INST]"]},
        model_kwargs={"n_gpu_layers": -1},
        verbose=True
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
    return index, llm

# Function to filter context based on a specified keyword
def filter_context(context, start_keyword):
    start_index = context.find(start_keyword)
    if start_index != -1:
        return context[start_index:]
    return context  # Return the original context if the keyword is not found

# Function to clean and filter text
def clean_text(text):
    cleaned_text = text.replace("<pad>", "").replace("<EOS>", "").replace("\n", " ")
    cleaned_text = " ".join(cleaned_text.split())
    cleaned_text = filter_context(cleaned_text, "Figure 3")
    return cleaned_text

# Load custom CSS
st.markdown('<style>' + open('style.css').read() + '</style>', unsafe_allow_html=True)

# Streamlit app header
st.markdown('<h2>Document Retrieval using RAG for Impetus Technologies</h2>', unsafe_allow_html=True)

# Upload section in Streamlit
uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

# If files are uploaded
if uploaded_files:
    # Save the uploaded files to the pdf_directory
    for uploaded_file in uploaded_files:
        file_path = os.path.join(pdf_directory, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

    st.success("Files uploaded successfully!")

    # Verify files in the directory
    files_in_directory = os.listdir(pdf_directory)
    st.write("Files in the directory:", files_in_directory)

    # If files are found, process them
    if files_in_directory:
        st.write("Processing documents...")
        index, llm = load_and_process_pdf(pdf_directory, text_embedding_model, llm_url)

        # Prompt Template (RAG)
        concise_template = Prompt("""
        Based on the following context from the document, provide a concise response to the user's query:
        Document Context:
        {context_str}
        User Query:
        {query_str}

        Response (one paragraph):
        """)

        # Query Engine
        query_engine = index.as_query_engine(streaming=True, llm=llm)

        # Chat UI
        st.write("Start querying the document:")
        user_query = st.text_input("Enter your query")

        if st.button("Submit Query"):
            # Generate response based on the user's query
            response = query_engine.query(user_query)
            st.write("Document Context Retrieved:")

            # Display retrieved document context and response
            for source_document in response.source_nodes:
                cleanText = clean_text(source_document.node.get_text())
                st.write(cleanText)

            st.write(f"Response: {response.response_text}")

# Add a footer
st.markdown('<footer><p>Streamlit app by Anushree Sharma</p></footer>', unsafe_allow_html=True)



Writing app.py


In [6]:
# Define the directory to save uploaded files
pdf_directory = '/content/sample_data'

import os
from pyngrok import ngrok
import streamlit as st
import subprocess
import time

# Set the authtoken for Ngrok
authtoken = '2j9MNrMbeZEpvOcfinlFoOaatLS_7qwJiUjjX4P1tPGgrcb7P'
ngrok.set_auth_token(authtoken)

# Start the Streamlit app
subprocess.Popen(["streamlit", "run", "app.py"])  # Using Popen to avoid blocking

# Allow some time for Streamlit to start
time.sleep(5)

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Keep the script running to maintain the Ngrok tunnel
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down...")
finally:
    ngrok.disconnect(public_url)
    ngrok.kill()


Streamlit app is live at: NgrokTunnel: "https://aa23-34-124-168-153.ngrok-free.app" -> "http://localhost:8501"
Shutting down...
